In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [3]:
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
#         print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "请求出错"         
        else:
            if ('geocodes' in temp):
                re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
def parse_gaode(df,address):
    # 解析高德
    list_gaode = get_receiver_lnglat_gaode(address)
    # df化(高德)
    df_gaode = pd.DataFrame(list_gaode)
    data_gaode = pd.concat([df,df_gaode],axis=1)
    print("data_gaode:" + str(data_gaode.shape))
    return data_gaode

In [5]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [6]:
# 获取百度地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [7]:
def parse_baidu(df,address):
    # 解析百度
    list_baidu = get_receiver_lnglat_baidu(address)
    # df化(百度)
    df_baidu = pd.DataFrame(list_baidu)
    data_baidu = pd.concat([df,df_baidu],axis=1)
    print("data_baidu:" + str(data_baidu.shape))
    return data_baidu

In [8]:
# ak码
# ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_gaode = "1c2682317206ccaa18ec2487bb45d880"
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [9]:
# 导入数据
date = "1902"
fileName = "京东线下(承运商).xlsx"
pinyin = "JDXX"
mainpath = "G:\\work\\basic\\carriers\\"
inputpath = mainpath + date + "\\"
data = pd.read_excel(inputpath + fileName)

In [10]:
# 导出数据路径
outpath_gaode = mainpath + date + "\\output\\gaode\\"
name_ANSI_gaode = date + "Info_lnglat_" + pinyin + "_gaode.xlsx"
name_UTF8_gaode = date + "Info_lnglat_" + pinyin + "_gaode.csv"

outpath_baidu = mainpath + date + "\\output\\baidu\\"
name_ANSI_baidu = date + "Info_lnglat_" + pinyin + "_baidu.xlsx"
name_UTF8_baidu = date + "Info_lnglat_" + pinyin + "_baidu.csv"


In [11]:
data["所在省"].value_counts()

广东     42045
山东     28201
河南     20616
江苏     14692
河北     13475
湖北     12805
北京     10939
湖南     10174
浙江      9755
四川      9663
辽宁      9344
上海      8384
陕西      7868
黑龙江     6934
广西      6105
山西      5956
安徽      5600
云南      5112
吉林      4932
内蒙古     4886
重庆      4018
江西      3996
福建      3579
贵州      3004
甘肃      2695
天津      2562
宁夏       577
海南       530
青海       439
西藏         2
新疆         1
Name: 所在省, dtype: int64

### 选取有效字段

In [12]:
df = data[["所在省","所在市","收货地址","订单编号","承运商","所在县"]]

In [13]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
df1 = df[df["所在省"].isin(["广东"])]
df2 = df[df["所在省"].isin(["山东"])]
df3 = df[df["所在省"].isin(["河南"])]

df4 = df[df["所在省"].isin(["江苏"])]
df5 = df[df["所在省"].isin(["河北"])]
df6 = df[df["所在省"].isin(["湖北"])]
df7 = df[df["所在省"].isin(["北京"])]
df8 = df[df["所在省"].isin(["湖南"])]

df9 = df[df["所在省"].isin(["浙江","上海"])]
df10 = df[df["所在省"].isin(["四川","云南","广西"])]
df11 = df[df["所在省"].isin(["辽宁","黑龙江","安徽"])]
df12 = df[df["所在省"].isin(["陕西","山西"])]

df13 = df[df["所在省"].isin(["吉林","内蒙古","重庆","江西","福建","贵州","甘肃","天津","宁夏","海南","青海","西藏","新疆"])]

### 文件分组并检查

In [18]:
print("data:" + str(data.shape))
print("df:" + str(df.shape))

data:(258889, 49)
df:(258889, 7)


In [ ]:
data1_gaode = parse_gaode(df1,df1["address_all"])

In [ ]:
data2_gaode = parse_gaode(df2,df2["address_all"])

In [ ]:
data3_gaode = parse_gaode(df3,df3["address_all"])

In [ ]:
data4_gaode = parse_gaode(df4,df4["address_all"])

In [ ]:
data5_gaode = parse_gaode(df5,df5["address_all"])

In [ ]:
data6_gaode = parse_gaode(df6,df6["address_all"])

In [ ]:
data7_gaode = parse_gaode(df7,df7["address_all"])

In [ ]:
data8_gaode = parse_gaode(df8,df8["address_all"])

In [ ]:
data9_gaode = parse_gaode(df9,df9["address_all"])

In [ ]:
data10_gaode = parse_gaode(df10,df10["address_all"])

In [ ]:
data11_gaode = parse_gaode(df11,df11["address_all"])

In [ ]:
data12_gaode = parse_gaode(df12,df12["address_all"])

In [ ]:
data13_gaode = parse_gaode(df13,df13["address_all"])

In [ ]:
data_gaode = pd.concat([data1_gaode,data2_gaode,data3_gaode,data4_gaode,data5_gaode,data6_gaode,data7_gaode,
                        data8_gaode,data9_gaode,data10_gaode,data11_gaode,data12_gaode,data13_gaode])

In [ ]:
# 合表
split_gaode = pd.DataFrame((x.split(',') for x in data_gaode['re_location_gaode']), index=data_gaode.index, columns=['re_siteLng_gaode','re_siteLat_gaode'])
data_gaode = pd.merge(data_gaode,split_gaode,left_index=True,right_index=True)


In [ ]:
# data_baidu = parse_baidu(df,df["address_all"])

In [ ]:
# 检查合并后结构
print("data:" + str(data.shape))
print("df:" + str(df.shape))
print("data_gaode:" + str(data_gaode.shape))
# print("data_baidu:" + str(data_baidu.shape))

In [ ]:
# data_gaode.to_csv(outpath_gaode + name_UTF8_gaode,index=False)
# data_baidu.to_csv(outpath_baidu + name_UTF8_baidu,index=False)

In [ ]:
data_gaode.head()